# Кластеризация судов

In [1]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic as GD

pd.options.mode.chained_assignment = None

%load_ext lab_black

In [2]:
# загрузка и очистка данных
df_ves = pd.read_csv("/home/sk27/repo/eremeev/test_projects/ves_test.csv").reset_index(
    drop=True
)
df_ves.drop_duplicates(inplace=True)

In [3]:
%%time

# пустой dataframe для аккумулирования результатов
df_result = pd.DataFrame()
# кластеризация в разрезе идентификатора орудия лова
for oper in list(set(df_ves.id_oper)):
    # фильтр по орудию лова
    df_ves_oper = df_ves.loc[(df_ves.id_oper == oper)]
    # присвоение каждому судну своего собственного номера кластера - на этом этапе число кластеров равно числу судов (или наоборот, как вам угодно)
    df_ves_oper["cluster"] = df_ves_oper.groupby("id_ves")["id_ves"].ngroup()
    # соединение данных по принипу декартова произведения ("каждое с каждым")
    df_cross = df_ves_oper.merge(df_ves_oper, how="cross")
    # рассчёт расстояний между судами в каждой паре
    df_cross["dist"] = df_cross.apply(
        lambda row: GD(
            (row["latitude_x"], row["longitude_x"]),
            (row["latitude_y"], row["longitude_y"]),
        ).km
        / 1.852,
        axis=1,
    )
    # фильтрафия полученных данных, оставляя строки, в которых расстояние не более 5 морских миль и коды судов различаются,
    # удаление дубликатов записей
    df_cross = (
        df_cross[
            (df_cross["dist"] <= 5) & (df_cross["id_ves_x"] != df_cross["id_ves_y"])
        ][["id_ves_x", "id_ves_y", "cluster_x", "cluster_y", "dist"]]
        .drop_duplicates()
        .reset_index(drop=True)
    )
    # создание словря, в котором каждому следующему коду судна сопоставляется код кластера
    # судна, находящегося от него на расстоянии не более 5 морских миль, если это судно уже
    # встречалось, иначе код кластера не изменяется
    cluster_dict = {}
    for index, row in df_cross.iterrows():
        if index == 0:
            cluster_dict[row["id_ves_x"]] = row["cluster_x"]
            cluster_dict[row["id_ves_y"]] = row["cluster_x"]
        else:
            if (
                row["id_ves_x"] in cluster_dict.keys()
                and row["id_ves_y"] not in cluster_dict.keys()
            ):
                cluster_dict[row["id_ves_y"]] = row["cluster_x"]
            elif row["id_ves_x"] not in cluster_dict.keys():
                cluster_dict[row["id_ves_x"]] = row["cluster_x"]
                cluster_dict[row["id_ves_y"]] = row["cluster_x"]
            else:
                pass
    # изменение значения кодов кластеров в первоначальном наборе данных для текущего орудия лова
    for key, value in cluster_dict.items():
        df_ves_oper["cluster"].where(
            ~(df_ves_oper["id_ves"] == key), other=value, inplace=True
        )
    # присоедниение данных к результирующему dataframe
    df_result = pd.concat([df_result, df_ves_oper]).reset_index(drop=True)
# формирование уникального идентификатора для каждого кластера
df_result["cluster"] = df_result.apply(
    lambda row: str(int(row["id_oper"])) + str(int(row["cluster"])), axis=1
)

CPU times: user 2.4 s, sys: 11 ms, total: 2.42 s
Wall time: 2.41 s


In [4]:
df_result.cluster.unique().shape[0]

93

In [17]:
pd.DataFrame(df_result.groupby("cluster").id_ves.count())

,id_ves
cluster,
110,3
113,1
114,1
1210,1
1214,1
...,...
205,1
206,1
207,1


In [13]:
# например, кластер "1258"
df_result.loc[df_result.cluster == "1258"]

,id_ves,latitude,longitude,id_oper,cluster
6,11767,58.68,156.26,12,1258
27,11533,58.65,156.16,12,1258
64,46271,58.63,156.21,12,1258
68,10019,58.65,156.18,12,1258


In [4]:
import antigravity

Found ffmpeg: /opt/yandex/browser-beta/libffmpeg.so
	avcodec: 3873892
	avformat: 3872868
	avutil: 3741797
FFmpeg version is too old. Need:
	avcodec: 3877988
	avformat: 3874916
	avutil: 3744868


find_ffmpeg failed, using the integrated library.


[190145:190145:0324/224549.274341:ERROR:isolated_origin_util.cc(74)] Ignoring port number in isolated origin: chrome://custo


Окно или вкладка откроются в текущем сеансе браузера.
